# RichHall

In [1]:
import sentinel1decoder
import pandas as pd
import numpy as np
import logging
import math
import cmath
import struct
import matplotlib.pyplot as plt
from matplotlib import colors
from scipy.interpolate import interp1d

In [2]:
inputfile = "/Data_large/marine/PythonProjects/SAR/sarpyx/data/S1A_S1_RAW__0SDV_20150107T054312_20150107T054337_004061_004E74_020B.SAFE/s1a-s1-raw-s-vh-20150107t054312-20150107t054337-004061-004e74.dat"
l0file = sentinel1decoder.Level0File(inputfile)

In [39]:
for selected_burst in range(1,160):
    burst = l0file.get_burst_metadata(selected_burst)
    print(f"Length of burst {selected_burst} is:", len(burst))

Length of burst 1 is: 8
Length of burst 2 is: 100
Length of burst 3 is: 50
Length of burst 4 is: 150
Length of burst 5 is: 20
Length of burst 6 is: 20
Length of burst 7 is: 60
Length of burst 8 is: 1851
Length of burst 9 is: 20
Length of burst 10 is: 1851
Length of burst 11 is: 20
Length of burst 12 is: 1851
Length of burst 13 is: 20
Length of burst 14 is: 1851
Length of burst 15 is: 20
Length of burst 16 is: 1851
Length of burst 17 is: 20
Length of burst 18 is: 1851
Length of burst 19 is: 20
Length of burst 20 is: 1851
Length of burst 21 is: 20
Length of burst 22 is: 1851
Length of burst 23 is: 20
Length of burst 24 is: 1851
Length of burst 25 is: 20
Length of burst 26 is: 1851
Length of burst 27 is: 20
Length of burst 28 is: 1851
Length of burst 29 is: 20
Length of burst 30 is: 1851
Length of burst 31 is: 20
Length of burst 32 is: 1851
Length of burst 33 is: 20
Length of burst 34 is: 1851
Length of burst 35 is: 20
Length of burst 36 is: 1851
Length of burst 37 is: 20
Length of burst 

KeyError: 64

In [40]:
burst

,Packet Version Number,Packet Type,Secondary Header Flag,PID,PCAT,Sequence Flags,Packet Sequence Count,Packet Data Length,Coarse Time,Fine Time,...,SWL,SAS SSB Flag,Polarisation,Temperature Compensation,Calibration Mode,Tx Pulse Number,Signal Type,Swap Flag,Swath Number,Number of Quads
Packet Number,,,,,,,,,,,,,,,,,,,,,
47583,0,0,1,65,12,3,14815,32322,1104644633,0.836098,...,0.000258,0,7,0,1,0,1,1,0,12862
47584,0,0,1,65,12,3,14816,32322,1104644633,0.836632,...,0.000258,0,7,0,1,0,1,1,0,12862
47585,0,0,1,65,12,3,14817,32322,1104644633,0.837166,...,0.000258,0,7,0,1,0,1,1,0,12862
47586,0,0,1,65,12,3,14818,32322,1104644633,0.837700,...,0.000258,0,7,0,1,0,1,1,0,12862
47587,0,0,1,65,12,3,14819,32322,1104644633,0.838234,...,0.000258,0,7,0,1,0,1,1,0,12862
47588,0,0,1,65,12,3,14820,32322,1104644633,0.838768,...,0.000258,0,7,0,1,0,1,1,0,12862
47589,0,0,1,65,12,3,14821,32322,1104644633,0.839302,...,0.000258,0,7,0,1,0,1,1,0,12862
47590,0,0,1,65,12,3,14822,32322,1104644633,0.839851,...,0.000258,0,7,0,1,0,1,1,0,12862


In [5]:
# # Decode the IQ data
# radar_data = l0file.get_burst_data(selected_burst)
# # Cache this data so we can retreive it more quickly next time we want it
# l0file.save_burst_data(selected_burst)

In [6]:
# Image sizes
len_range_line = 25724 # radar_data.shape[1]
len_az_line = 56130 # radar_data.shape[0]

# Tx pulse parameters
c = sentinel1decoder.constants.SPEED_OF_LIGHT_MPS
RGDEC = selection["Range Decimation"].unique()[0]
PRI = selection["PRI"].unique()[0]
rank = selection["Rank"].unique()[0]
suppressed_data_time = 320/(8*sentinel1decoder.constants.F_REF)
range_start_time = selection["SWST"].unique()[0] + suppressed_data_time
wavelength = sentinel1decoder.constants.TX_WAVELENGTH_M

# Sample rates
range_sample_freq = sentinel1decoder.utilities.range_dec_to_sample_rate(RGDEC)
range_sample_period = 1/range_sample_freq
az_sample_freq = 1 / PRI
az_sample_period = PRI

# Fast time vector - defines the time axis along the fast time direction
sample_num_along_range_line = np.arange(0, len_range_line, 1)
fast_time_vec = range_start_time + (range_sample_period * sample_num_along_range_line)

# Slant range vector - defines R0, the range of closest approach, for each range cell
slant_range_vec = ((rank * PRI) + fast_time_vec) * c/2
    
# Axes - defines the frequency axes in each direction after FFT
SWL = len_range_line/range_sample_freq
az_freq_vals = np.arange(-az_sample_freq/2, az_sample_freq/2, 1/(PRI*len_az_line))
range_freq_vals = np.arange(-range_sample_freq/2, range_sample_freq/2, 1/SWL)
 
# # Spacecraft velocity - numerical calculation of the effective spacecraft velocity
# ecef_vels = l0file.ephemeris.apply(lambda x: math.sqrt(x["X-axis velocity ECEF"]**2 + x["Y-axis velocity ECEF"]**2 +x["Z-axis velocity ECEF"]**2), axis=1)
# velocity_interp = interp1d(l0file.ephemeris["POD Solution Data Timestamp"].unique(), ecef_vels.unique(), fill_value="extrapolate")
# x_interp = interp1d(l0file.ephemeris["POD Solution Data Timestamp"].unique(), l0file.ephemeris["X-axis position ECEF"].unique(), fill_value="extrapolate")
# y_interp = interp1d(l0file.ephemeris["POD Solution Data Timestamp"].unique(), l0file.ephemeris["Y-axis position ECEF"].unique(), fill_value="extrapolate")
# z_interp = interp1d(l0file.ephemeris["POD Solution Data Timestamp"].unique(), l0file.ephemeris["Z-axis position ECEF"].unique(), fill_value="extrapolate")
# space_velocities = selection.apply(lambda x: velocity_interp(x["Coarse Time"] + x["Fine Time"]), axis=1).to_numpy().astype(float)

# x_positions = selection.apply(lambda x: x_interp(x["Coarse Time"] + x["Fine Time"]), axis=1).to_numpy().astype(float)
# y_positions = selection.apply(lambda x: y_interp(x["Coarse Time"] + x["Fine Time"]), axis=1).to_numpy().astype(float)
# z_positions = selection.apply(lambda x: z_interp(x["Coarse Time"] + x["Fine Time"]), axis=1).to_numpy().astype(float)

# position_array = np.transpose(np.vstack((x_positions, y_positions, z_positions)))

# a = sentinel1decoder.constants.WGS84_SEMI_MAJOR_AXIS_M
# b = sentinel1decoder.constants.WGS84_SEMI_MINOR_AXIS_M
# H = np.linalg.norm(position_array, axis=1)
# W = np.divide(space_velocities, H)
# lat = np.arctan(np.divide(position_array[:, 2], position_array[:, 0]))
# local_earth_rad = np.sqrt(
#     np.divide(
#         (np.square(a**2 * np.cos(lat)) + np.square(b**2 * np.sin(lat))),
#         (np.square(a * np.cos(lat)) + np.square(b * np.sin(lat)))
#     )
# )
# cos_beta = (np.divide(np.square(local_earth_rad) + np.square(H) - np.square(slant_range_vec[:, np.newaxis]) , 2 * local_earth_rad * H))
# ground_velocities = local_earth_rad * W * cos_beta

# effective_velocities = np.sqrt(space_velocities * ground_velocities)

# D = np.sqrt(
#     1 - np.divide(
#         wavelength**2 * np.square(az_freq_vals),
#         4 * np.square(effective_velocities)
#     )
# ).T

# # We're only interested in keeping D, so free up some memory by deleting these large arrays.
# del effective_velocities
# del ground_velocities
# del cos_beta
# del local_earth_rad
# del H
# del W
# del lat

In [9]:
# Create replica pulse
TXPSF = selection["Tx Pulse Start Frequency"].unique()[0]
TXPRR = selection["Tx Ramp Rate"].unique()[0]
TXPL = selection["Tx Pulse Length"].unique()[0]
num_tx_vals = int(TXPL*range_sample_freq)
print(f'num_tx_vals: {num_tx_vals}')


tx_replica_time_vals = np.linspace(-TXPL/2, TXPL/2, num=num_tx_vals)
phi1 = TXPSF + TXPRR*TXPL/2
phi2 = TXPRR/2
tx_replica = np.exp(2j * np.pi * (phi1*tx_replica_time_vals + phi2*tx_replica_time_vals**2))


print("REPLICA LENGTH: ", len(tx_replica))

# Create range filter from replica pulse
range_filter = np.zeros(len_range_line, dtype=complex)
index_start = np.ceil((len_range_line-num_tx_vals)/2)-1
index_end = num_tx_vals+np.ceil((len_range_line-num_tx_vals)/2)-2
range_filter[int(index_start):int(index_end+1)] = tx_replica
range_filter = np.conjugate(np.fft.fft(range_filter))



# Print all variable values
# print(f'PRI: {PRI}')
# print(f'RGDEC: {RGDEC}')
# print(f'SWL: {SWL}')
# print(f'TXPL: {TXPL}')
# print(f'TXPRR: {TXPRR}')
# print(f'TXPSF: {TXPSF}')
# print(f'az_freq_vals: {az_freq_vals}')
# print(f'az_sample_freq: {az_sample_freq}')
# print(f'az_sample_period: {az_sample_period}')
# print(f'c: {c}')
# print(f'fast_time_vec: {fast_time_vec}')
print(f'index_end: {index_end}')
print(f'index_start: {index_start}')
# print(f'inputfile: {inputfile}')
# print(f'l0file: {l0file}')
# print(f'len_az_line: {len_az_line}')
# print(f'len_range_line: {len_range_line}')
# print(f'num_tx_vals: {num_tx_vals}')
# print(f'phi1: {phi1}')
# print(f'phi2: {phi2}')
# print(f'range_filter: {range_filter}')
# print(f'range_freq_vals: {range_freq_vals}')
# print(f'range_sample_freq: {range_sample_freq}')
# print(f'range_sample_period: {range_sample_period}')
# print(f'range_start_time: {range_start_time}')
# print(f'rank: {rank}')
# print(f'sample_num_along_range_line: {sample_num_along_range_line}')
# print(f'selected_burst: {selected_burst}')
# print(f'selection: {selection}')
# print(f'slant_range_vec: {slant_range_vec}')
# print(f'suppressed_data_time: {suppressed_data_time}')
# print(f'tx_replica: {tx_replica}')
# print(f'tx_replica_time_vals: {tx_replica_time_vals}')
# print(f'wavelength: {wavelength}')




# Apply filter
# radar_data = np.multiply(radar_data, range_filter)

# del range_filter
# del tx_replica

num_tx_vals: 4549
REPLICA LENGTH:  4549
index_end: 15135.0
index_start: 10587.0


In [8]:
metadata = l0file.get_burst_metadata(selected_burst)
print(metadata.iloc[0])

Packet Version Number                        0.000000e+00
Packet Type                                  0.000000e+00
Secondary Header Flag                        1.000000e+00
PID                                          6.500000e+01
PCAT                                         1.200000e+01
Sequence Flags                               3.000000e+00
Packet Sequence Count                        4.080000e+02
Packet Data Length                           1.730600e+04
Coarse Time                                  1.398687e+09
Fine Time                                    6.424484e-01
Sync                                         8.922707e+08
Data Take ID                                 2.188396e+08
ECC Number                                   1.100000e+01
Test Mode                                    0.000000e+00
Rx Channel ID                                1.000000e+00
Instrument Configuration ID                  7.000000e+00
Sub-commutated Ancilliary Data Word Index    2.500000e+01
Sub-commutated